In [ ]:
from itertools import count
from collections import Counter
import numpy as np
from scipy import stats

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def sequence(
    start: int, num_bits: int, f: "int -> bit", max_steps: int
):
    xs = [start]
    
    for i in range(max_steps):
        new = (xs[-1] >> 1) | (f(xs[-1]) << num_bits)
        
        if new == start:
#             print(f"loop found after {i+1} steps")
            break
            
        xs.append(new)
    
    return xs


n = 33
xs = sequence(
    start=((1 << n) | 1),
    num_bits=n,
    f=lambda x: (x ^ (x >> 1) ^ (x >> 2) ^ (x >> 7)) & 1,
#     f=lambda x: (x ^ (x >> 1)) & 1,
    max_steps=1_000_000
)
# print(xs)
cnt = Counter([x & 1 for x in xs])
print(cnt)

p_value = stats.binom_test(
    cnt[1], n=(cnt[0] + cnt[1]), p=0.5, alternative='two-sided'
)
print(f"{p_value=:.5f}")

# sns.histplot(np.log2(xs))
# plt.xlabel('Bits')

In [ ]:
d = {}

ns = np.arange(2, 127)
ys = np.zeros_like(ns, dtype=float)
ps = np.zeros_like(ns, dtype=float)

for i, n in tqdm(enumerate(ns), total=len(ns)):
    xs = sequence(
        start=((1 << n) | 1),
        num_bits=n,
        f=lambda x: (x ^ (x >> 1) ^ (x >> 2) ^ (x >> 7)) & 1,
        max_steps=1_000_000
    )
    cnt = Counter([x & 1 for x in xs])
    ys[i] = cnt[1] / (cnt[0] + cnt[1])
    ps[i] = stats.binom_test(
        cnt[1], n=(cnt[0] + cnt[1]), p=0.5, alternative='two-sided'
    )

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.plot(ns, ys, linestyle='--', marker='.')
plt.subplot(1, 2, 2)
plt.plot(ns, ps, linestyle='--', marker='.')
pass